<a href="https://colab.research.google.com/github/ilyashumilov/Alllocation/blob/main/Z_Image_Turbo_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI
!pip install -r requirements.txt

!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors

fatal: destination path 'ComfyUI' already exists and is not an empty directory.
/content/ComfyUI
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
204bf5|OK  |       0B/s|/content/ComfyUI/models/diffusion_models/z-image-turbo-fp8-e4m3fn.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
523965|OK  |       0B/s|/content/ComfyUI/models/clip/qwen_3_4b.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
769806|OK  |       0B/s|/content/ComfyUI/models/vae/ae.safetensors

Status Legend:
(OK):download completed.


In [3]:
%cd /content/ComfyUI

import os, random, time
import torch
import numpy as np
from PIL import Image

from nodes import NODE_CLASS_MAPPINGS

# Загружаем нужные ноды (добавляем LoadImage и VAEEncode)
UNETLoader    = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader    = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader     = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode= NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler      = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode     = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
LoadImage     = NODE_CLASS_MAPPINGS["LoadImage"]()          # ← новая
VAEEncode     = NODE_CLASS_MAPPINGS["VAEEncode"]()          # ← новая

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae  = VAELoader.load_vae("ae.safetensors")[0]

@torch.inference_mode()
def generate(input_dict):
    tmp_dir = "/content/ComfyUI/output"
    os.makedirs(tmp_dir, exist_ok=True)

    values = input_dict["input"]

    positive_prompt = values['positive_prompt']
    negative_prompt = values.get('negative_prompt', "")
    seed            = values.get('seed', 0)
    steps           = values.get('steps', 9)
    cfg             = values.get('cfg', 1.0)
    sampler_name    = values.get('sampler_name', "euler")
    scheduler       = values.get('scheduler', "simple")
    denoise         = values.get('denoise', 0.35)           # ← КЛЮЧ! 0.25–0.45 для лица
    width           = values.get('width', 1024)
    height          = values.get('height', 1024)
    batch_size      = values.get('batch_size', 1)

    # ← Загрузка исходного изображения (обязательно!)
    input_image_path = values.get('/content/1.jpeg')            # путь, например "/content/ref_face.jpg"
    if not input_image_path or not os.path.exists(input_image_path):
        raise ValueError("Укажи существующий путь input_image!")

    image, _ = LoadImage.load_image(input_image_path)       # image — это тензор IMAGE
    latent_image = VAEEncode.encode(vae, image)[0]          # ← вместо EmptyLatentImage!

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)

    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]

    samples = KSampler.sample(
        model=unet,
        seed=seed,
        steps=steps,
        cfg=cfg,
        sampler_name=sampler_name,
        scheduler=scheduler,
        positive=positive,
        negative=negative,
        latent_image=latent_image,                          # ← подаём уже зашумлённый latent
        denoise=denoise
    )[0]

    decoded = VAEDecode.decode(vae, samples)[0].detach().cpu()
    # Берём первое изображение из батча
    img_array = (decoded[0].numpy() * 255).astype(np.uint8)
    result_img = Image.fromarray(img_array)

    output_path = f"{tmp_dir}/z_turbo_img2img.png"
    result_img.save(output_path)

    return output_path

/content/ComfyUI


In [1]:
input_dict = {
  "input": {
    "positive_prompt": "A girl standing in front of a large bathroom mirror, capturing a selfie with her right hand holding a yellow phone case featuring a small cartoon character charm dangling from it, the phone positioned at eye level and angled towards the mirror to frame her upper body and midriff in the reflection, her left hand raised near her shoulder forming a precise peace sign with the index and middle fingers extended straight while the other fingers are curled in, thumb tucked against the palm, her pose slightly angled with one hip shifted subtly to the side for a three-quarter view, emphasizing her slender and toned figure with a narrow waist, defined abdominal lines, and smooth curves from ribs to hips, her gaze directed straight at the phone screen as if checking the shot, lips softly pressed together in a neutral, relaxed expression without smile or pout, mouth closed evenly, wearing a pale pink spaghetti-strap camisole of silky satin material that clings lightly to her form with thin adjustable straps over the shoulders and a small decorative bow at the center of the bustline, paired with low-rise pale pink bottoms of similar silky fabric sitting just below the navel, the location is a modern bathroom with light gray marble walls and countertop veined in white, a partially open door in the background revealing a hint of wooden flooring, a blue-and-white striped towel draped over a rack to the right, and soft natural daylight filtering in from a window creating a calm, intimate atmosphere with subtle shadows and no harsh lighting, the overall style of the photo resembling a casual snapshot taken on an ordinary smartphone without any professional editing or staging, maximally realistic with authentic human skin texture showing natural pores, subtle imperfections, and no artificial smoothness or plastic appearance. The face of the model should be taken from the original input file, with the face absent in the description.",
    "negative_prompt": "blurry, deformed, cartoon, plastic, oversmoothed, bad anatomy",
    "input_image": "/content/1.jpeg",     # ← твой файл!
    "denoise": 0.35,                                 # 0.25 = почти копия, 0.4 = заметные изменения стиля/одежды
    "steps": 10,
    "cfg": 1.0,
    "seed": 123456789,
    "width": 1024,
    "height": 1024,
    "batch_size": 1
  }
}

output_path = generate(input_dict)
display(Image.open(output_path))                     # или Image.open(output_path)

NameError: name 'generate' is not defined